- https://ddosdb.org/attack-trace/80f37aac99809f9890e3d5dd9993a2a6
- https://ddosdb.org/attack-trace/02f2204ded5031080840041c9d22d5ed
- https://ddosdb.org/attack-trace/f2724bf2d516c91e8373f7153a595ff1_4

In [1]:
import pandas as pd 
import numpy as np
import glob

In [2]:
def processing_pcap_ttl(pcap_file):
    !tshark -n -r $pcap_file -E separator=\;  -E header=y -T fields -e ip.src -e ip.ttl  > pcap.txt
    df = pd.read_csv('pcap.txt', error_bad_lines=False, sep=';')
    
    try:
        df['ip.src'] = df['ip.src'].apply(lambda x: x.split(',')[0])
    except:
        pass
    
    try:
        df['ip.ttl'] = df['ip.ttl'].apply(lambda x: int(x.split(',')[0]))
    except:
        pass
    
    df_summary = df.groupby(['ip.src'])['ip.ttl'].agg([np.ptp]).reset_index()
    summary = df_summary['ptp'].value_counts()
    return summary

## Reading all files

In [3]:
pcap_files = glob.glob("../*.pcap")
pcap_files

['../02f2204ded5031080840041c9d22d5ed.pcap',
 '../f2724bf2d516c91e8373f7153a595ff1_4.pcap',
 '../80f37aac99809f9890e3d5dd9993a2a6.pcap']

## LOOP TO PROCESS AND CONCAT EVERYTHING

In [4]:
df = pd.DataFrame()

for pcap_file in pcap_files:
    summary = processing_pcap_ttl(pcap_file).to_frame().T.reset_index()
    summary.at[0, 'index'] = pcap_file.replace('../','').replace('.pcap','')
    df = pd.concat([summary,df], sort=False)

df = df.fillna(0).set_index('index')
df = df.reindex(columns=sorted(df.columns))
df['total_ips'] = df.sum(axis=1)
df

,0,1,2,3,4,5,8,13,63,192,total_ips
index,,,,,,,,,,,
80f37aac99809f9890e3d5dd9993a2a6,22,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,32.0
f2724bf2d516c91e8373f7153a595ff1_4,1043,84,16.0,24.0,66.0,0.0,0.0,1.0,0.0,0.0,1234.0
02f2204ded5031080840041c9d22d5ed,9098,11,2.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,9116.0


In [5]:
pd.options.display.float_format = '{:.2f}%'.format
df_percent = df.div(df['total_ips'], axis=0)*100
df_percent

,0,1,2,3,4,5,8,13,63,192,total_ips
index,,,,,,,,,,,
80f37aac99809f9890e3d5dd9993a2a6,68.75%,28.12%,0.00%,0.00%,0.00%,0.00%,3.12%,0.00%,0.00%,0.00%,100.00%
f2724bf2d516c91e8373f7153a595ff1_4,84.52%,6.81%,1.30%,1.94%,5.35%,0.00%,0.00%,0.08%,0.00%,0.00%,100.00%
02f2204ded5031080840041c9d22d5ed,99.80%,0.12%,0.02%,0.01%,0.01%,0.01%,0.00%,0.00%,0.01%,0.01%,100.00%
